In [3]:
!python -V

Python 3.12.1


In [8]:
!pip install mlflow

In [1]:
import pandas as pd

In [2]:
import pickle

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

In [5]:
import mlflow


mlflow.set_tracking_uri("http://localhost:8080")
mlflow.set_experiment("orchestration-hw")

<Experiment: artifact_location='mlflow-artifacts:/930827882230850165', creation_time=1749293889573, experiment_id='930827882230850165', last_update_time=1749293889573, lifecycle_stage='active', name='orchestration-hw', tags={}>

In [9]:
def read_dataframe():
    df = pd.read_parquet("./yellow_tripdata_2023-03.parquet")
    print(df.shape)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    numerical = ['trip_distance']
    
    df[categorical] = df[categorical].astype(str)

    dicts = df[categorical + numerical].to_dict(orient='records')
    dv = DictVectorizer()
    X_train = dv.fit_transform(dicts)

    return df, X_train, dv

def train_model(X_train, y_train):

    print("train model")
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    print("Intercept:", lr.intercept_)

    return lr

def run():
    df, X_train, dv = read_dataframe()
    print(df.shape)
    
    target = 'duration'
    y_train = df[target].values

    return X_train, y_train

    # train_model(X_train, y_train)
    # print(f"MLflow run_id: {run_id}")
    # return run_id

In [7]:
X_train, y_train = run()

(3403766, 19)
(3316216, 20)


In [10]:
lr = train_model(X_train, y_train)

train model
Intercept: 23.847794585831632


In [11]:
with mlflow.start_run() as run:
    # Log parameters and metrics
    mlflow.log_param("fit_intercept", lr.fit_intercept)
    mlflow.log_metric("intercept", lr.intercept_)

    # Log and register the model
    mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="model",
        registered_model_name="linear-regression-model"  # this registers it
    )


2025/06/07 12:17:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'linear-regression-model'.
2025/06/07 12:17:25 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: linear-regression-model, version 1


🏃 View run amazing-mouse-949 at: http://localhost:8080/#/experiments/930827882230850165/runs/e56f258e4e6d477fb2ca6125dd79fe1e
🧪 View experiment at: http://localhost:8080/#/experiments/930827882230850165


Created version '1' of model 'linear-regression-model'.
